# Final Project

## Data Preprocessing

In [1]:
import requests
import bs4
import json
import pandas as pd
import geopandas as gpd
import psycopg2
from shapely.geometry import Point
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, text,Float, Date, DateTime
from sqlalchemy.schema import Index
from geoalchemy2 import Geometry

In [2]:
data_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
data_tree = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json'
headers={"X-App-Token": "SRW3mUlBC1WT0KJr9H06yMUfF"}

In [4]:
params = {
    '$where': "created_date between '2015-01-01T00:00:00' and '2023-12-31T23:59:59' ",
               
    '$select': "created_date, incident_zip, complaint_type, latitude, longitude ",
    
    '$limit': 999999999
}

headers={"X-App-Token": "SRW3mUlBC1WT0KJr9H06yMUfF"}

response_311 = requests.get(data_311, params=params, headers=headers)
json_311 = response_311.json()
df_311 = pd.DataFrame.from_records(json_311).dropna()
df_311['geometry'] = df_311.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
df_311

,created_date,incident_zip,complaint_type,latitude,longitude,geometry
0,2023-12-01T12:00:00.000,10465,Derelict Vehicles,40.84384365721464,-73.82454971742995,POINT (-73.82454971742995 40.84384365721464)
1,2023-12-01T01:03:37.000,10308,Noise - Residential,40.56203453710598,-74.15721538110013,POINT (-74.15721538110013 40.56203453710598)
2,2023-12-01T01:03:29.000,11207,Noise - Vehicle,40.68471967420527,-73.90681258883566,POINT (-73.90681258883566 40.68471967420527)
3,2023-12-01T01:03:27.000,10467,Noise - Residential,40.87631257939169,-73.86697759703779,POINT (-73.86697759703779 40.87631257939169)
4,2023-12-01T01:03:20.000,10024,Encampment,40.78515581987176,-73.97900165965613,POINT (-73.97900165965613 40.78515581987176)
...,...,...,...,...,...,...
24919305,2015-01-01T00:00:00.000,10457,PAINT/PLASTER,40.84958298506529,-73.9039755233459,POINT (-73.9039755233459 40.84958298506529)
24919306,2015-01-01T00:00:00.000,10039,Rodent,40.82496024161445,-73.94076840452809,POINT (-73.94076840452809 40.82496024161445)
24919307,2015-01-01T00:00:00.000,11418,HEAT/HOT WATER,40.7047906049207,-73.81993121069071,POINT (-73.81993121069071 40.7047906049207)
24919308,2015-01-01T00:00:00.000,10019,HEAT/HOT WATER,40.76710752891153,-73.98631416418456,POINT (-73.98631416418456 40.76710752891153)


In [27]:
params2 = {
    '$select': "tree_id, status, health, zipcode, spc_common, latitude, longitude ",
    
    '$limit': 999999999
}

response_tree = requests.get(data_tree, params=params2, headers=headers)
json_tree = response_tree.json()
df_tree = pd.DataFrame.from_records(json_tree).dropna()
df_tree['geometry'] = df_tree.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
df_tree

,tree_id,status,health,zipcode,spc_common,latitude,longitude,geometry
0,180683,Alive,Fair,11375,red maple,40.72309177,-73.84421522,POINT (-73.84421522 40.72309177)
1,200540,Alive,Fair,11357,pin oak,40.79411067,-73.81867946,POINT (-73.81867946 40.79411067)
2,204026,Alive,Good,11211,honeylocust,40.71758074,-73.9366077,POINT (-73.9366077 40.71758074)
3,204337,Alive,Good,11211,honeylocust,40.71353749,-73.93445616,POINT (-73.93445616 40.71353749)
4,189565,Alive,Good,11215,American linden,40.66677776,-73.97597938,POINT (-73.97597938 40.66677776)
...,...,...,...,...,...,...,...,...
683783,155433,Alive,Good,11211,pin oak,40.71321078,-73.95494401,POINT (-73.95494401 40.71321078)
683784,183795,Alive,Good,11375,Kentucky yellowwood,40.71519444,-73.8566502,POINT (-73.8566502 40.71519444)
683785,166161,Alive,Good,10314,red maple,40.62076153,-74.13651724,POINT (-74.13651724 40.62076153)
683786,184028,Alive,Good,10457,red maple,40.85082819,-73.90311472,POINT (-73.90311472 40.85082819)


In [45]:
columns_to_drop = ['SizeRank', 'RegionID', 'RegionType', 'StateName', 'State','Metro']

df_rent = pd.read_csv('zillow_rent_data.csv').drop(columns=columns_to_drop)
mask = df_rent['City'] == 'New York'
zillow_rent = df_rent[mask]
zillow_rent

,RegionName,City,CountyName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
4,11385,New York,Queens County,NaN,2087.527084,NaN,2149.924252,2166.263698,2148.992886,2190.098591,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783
6,11208,New York,Kings County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2508.670432,2588.030194,2613.790654,2585.561351,2633.200754,2672.038493,2806.918757,2765.224364,2737.547470,2728.733333
12,11236,New York,Kings County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2285.460026,2362.500000
13,10467,New York,Bronx County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2145.642295,2155.617718,2172.346611,2160.962748,2110.533203,2180.323655,2276.372290,2334.204728,2353.686402,2423.888889
14,11373,New York,Queens County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2199.459063,2255.604528,2262.101623,2271.514956,2250.182334,2231.959479,2257.413993,2247.592851,2302.557354,2292.994444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6550,10282,New York,New York County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7058.613416,7143.347680,7408.940270,7459.080381,7512.509440,7452.333390,7572.806191,7611.834625,7444.236260,7347.458333
6561,11109,New York,Queens County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4247.334177,4159.759749,4157.528579,4201.199144,4294.386817,4343.065214,4357.893069,4445.207586,4490.367074,4529.358974
6644,10006,New York,New York County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3936.629997,3747.416799,3693.121513,3812.835588,3920.619972,4025.134283,4043.838704,4035.676503,4042.831474,4060.096154
6695,10162,New York,New York County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4871.181752,5007.415824,NaN,4984.693932,5011.666667


In [46]:
zillow_rent = zillow_rent.melt(id_vars=list(zillow_rent.columns)[:3], var_name='Date', value_name='Rent').dropna()
zillow_rent

,RegionName,City,CountyName,Date,Rent
5,11226,New York,Kings County,2015-01-31,1944.609891
7,10025,New York,New York County,2015-01-31,3068.951823
13,11206,New York,Kings County,2015-01-31,2482.829299
14,11221,New York,Kings County,2015-01-31,2125.738807
20,11235,New York,Kings County,2015-01-31,1687.789898
...,...,...,...,...,...
15220,10282,New York,New York County,2023-09-30,7347.458333
15221,11109,New York,Queens County,2023-09-30,4529.358974
15222,10006,New York,New York County,2023-09-30,4060.096154
15223,10162,New York,New York County,2023-09-30,5011.666667


In [59]:
columns_to_drop2 = ['BLDGZIP', 'POPULATION', 'AREA', 'STATE', 'ST_FIPS','CTY_FIPS', 'SHAPE_AREA', 'SHAPE_LEN','URL','PO_NAME','COUNTY']
zipcode = gpd.read_file('nyc_zipcodes.shp').drop(columns=columns_to_drop2)
zipcode = zipcode.to_crs(epsg=4632)
zipcode

,ZIPCODE,geometry
0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."
...,...,...
258,10310,"POLYGON ((-74.12065 40.64104, -74.12057 40.641..."
259,11693,"POLYGON ((-73.84076 40.62536, -73.84306 40.627..."
260,11249,"POLYGON ((-73.95805 40.72442, -73.95772 40.724..."
261,10162,"POLYGON ((-73.95133 40.76931, -73.95165 40.769..."


## Storing Data

## Understanding Data

## Visualizing Data

## Extra Credit